# Introduction

The goal of this notebook is to produce two stars of equal mass orbiting eachother in Newtonian physics. It should be trivially extensible to stars that are not equal mass and to solar systems with one planet. 

In newtonian gravity in space (as opposed to on the surface of the Earth, where the size of the Earth is so overwhelming compared to how far you walk that the gravitational field doesn't appear to change), stars orbit around eachother in circles or ellipses. It is also possible to have parabolic or hyperbolic orbits that are unbound. Gravity is a central force, and forces come in equal and opposite pairs. As a result, for equal mass motion, the stars motions are symmetric about a center point. 

While it is ideal to be able to compute hyperbolic, parabolic, and elliptical orbits, initially I will focus on circular orbits. 

Why would I do this, you might ask, given that circular orbits can be trivially computed with a little algebra in about the sixth week of the first year of freshman physics at the college level? 
- Because once I have a differential equation solver I can move on to orbits of more than two stars, which is interesting for stellar physics, globular clusters, exoplanets, and galactic dynamics. 
- Because it will help me demo my code in a simpler context than general relativity. 
- Because it is fun. 
- Because it is good coding practice and I need to pass a code test in industry and I've been out of the field for a year while I worked as a volunteer teacher. 

Here's the computation of the initial conditions, which is also a solution for the whole orbit for circular orbits.

Centripital force F_1=m_1*v^2/r
Newtonain Gravitational Force F_1=G*m_1*m_2/r^2

Setting the two forces equal, v=sqrt(m_2/r)

By Newtons laws m_1*a_1=F_1

Setting that equal to the gravitational force

a_1=G*m_2/r^2

And that's all you need unless you transform to the reduced mass coordinate system where one mass is held fixed at the center. Normally both masses have to be evolved around eachother with equal and opposite force pairs. I use this computation for my initial conditions, but solve the orbit numerically using a differential equation solver, either Euler or Runga Kutta 4th order.

The orbits should conserve energy. E=1/2*m1*v1^2+ 1/2*m2*v2*2 -G*m1*m2/r^2

The orbits should conserve angular momentum. L=m1*v1*r+m2*v2*r

The orbits should preserve orbital radius if they're circular. 

The focus of an ellipse, hyperbola, or parabola should be at the center of mass of the orbit (half way between two bodies of equal mass).

# Differential equation solvers

In [3]:
#from pip._internal import main as pip_main
#except ImportError:
#    from pip._internal import main as pip_main

#package_to_install = "numpy"
#try:
#    pip_main(["install", "--user", package_to_install])
#except SystemExit as e:
#    print(e)

The Newtonian Gravitational Force depends upon the masses of the two objects and the square of the distances between them. When one astronomical body acts upon another with force F12, the second body acts upon the first with equal and opposite force F21=-F12. Gravitational Force follows an inverse square law with regard to distance scaling that is exact in Newtonian gravity.

In [4]:
import numpy as np
import matplotlib.pyplot as plt

def NewtonianForce(mass1,mass2, r):
    return (6.67408*10**-11)*mass1*mass2/r/r;

To compute an orbit, it is necessary to solve a second order differential equation. 

F=ma

For a two body problem:

F12x=m1 d^2 x1/dt^2

This can be decomposed into two coupled differential equations.

F12x= m1 d v1/dt

v1= d x1/dt

There is also a y component and a z component. 
It is also necessary to track the position of the second particle.

Recall that once a force F12 is split into its components in the x and y direction, F12 will no longer exactly be given by the NewtonainForce equation given above but rather the magnitude of F12x, F12y, and F12z combined will equal that total. The nagnitude of a vector is found by squaring and summing the components then taking the square roots. So, sqrt(F12x^2+F12y^2+F12z^2)=FNewtonianForce

Assume, for the purposes of a two body problem, the orbit is confined to the x-y plane. 

Define polar coordinates phi in the x-y plane beginning from the x axis and proceeding toward the y axis. Then F12x=F12 cos phi and F12y= F12 sin phi.

F12x=x/sqrt(x^2+y^2)*F12

F12y = y/sqrt(x^2+y^2)*F12

# Numerical solutions to differential equations

To integrate a differential equation numerically, simply differentiate both of the two coupled equations, one time steop at a time, numerically. But how is this done numerically? There are a couple of ways. First consider how a derivative is defined. A derivative is a limit of a difference divided by a timestep. The euler derivative makes this definition discrete for small time steps. 

In [5]:
def euler(h,t, x,y,z,f,debugprint):
    xstep = (f(t, x, y,z)+f(t+h,x+h,y,z))/2*h
    xnew=x+xstep
    if(debugprint):
        print(h,t,x,xstep,xnew)
    return t+h,xnew

The euler routine essentially just finds the slope in a small region of the graph. 

The Runga Kutta numerical derivative is a fourth order approximation to the derivative that accounts for the curvature and is better for something like an orbit that fundamentally has curvature to it where orvershoot near a minima may matter. 

# Verify Euler numerically

In [6]:
def polynomial(x,power,a):
    p=a*x**power
    #print(pi)
    return p
    

def polymaker1(t,x,y,z,power,a):
    def poly(t,x,y,z):
        return polynomial(x,power,a)
    return poly

#exact solution to pow=1 is lnx

In [7]:
import numpy as np
t=0.
x=1.
y=1.
z=1.
print(t,x)
polyfn=polymaker1(t,x,y,z,1,1.0)

0.0 1.0


In [8]:
outp=polyfn(t,x,y,z) #this is correct: x^2
print(outp)

1.0


In [9]:
eulerpolyout=euler(.01,t,x,y,z,polyfn,True)
print(eulerpolyout)

0.01 0.0 1.0 0.01005 1.01005
(0.01, 1.01005)


In [10]:
dt=0.01
numsteps=10
xvals=[]
tvals=[]
for i in np.arange(1,numsteps):
    #t=0.+i*dt
    t,x=euler(dt,t,x,y,z,polyfn,True)
    print(x)
    xvals.append(x)
    tvals.append(t)
print(xvals)
print(tvals)

0.01 0.0 1.0 0.01005 1.01005
1.01005
0.01 0.01 1.01005 0.0101505 1.0202004999999998
1.0202004999999998
0.01 0.02 1.0202004999999998 0.010252005 1.0304525049999997
1.0304525049999997
0.01 0.03 1.0304525049999997 0.010354525049999999 1.0408070300499996
1.0408070300499996
0.01 0.04 1.0408070300499996 0.010458070300499998 1.0512651003504996
1.0512651003504996
0.01 0.05 1.0512651003504996 0.010562651003504998 1.0618277513540046
1.0618277513540046
0.01 0.060000000000000005 1.0618277513540046 0.010668277513540048 1.0724960288675447
1.0724960288675447
0.01 0.07 1.0724960288675447 0.010774960288675448 1.0832709891562202
1.0832709891562202
0.01 0.08 1.0832709891562202 0.010882709891562202 1.0941536990477825
1.0941536990477825
[1.01005, 1.0202004999999998, 1.0304525049999997, 1.0408070300499996, 1.0512651003504996, 1.0618277513540046, 1.0724960288675447, 1.0832709891562202, 1.0941536990477825]
[0.01, 0.02, 0.03, 0.04, 0.05, 0.060000000000000005, 0.07, 0.08, 0.09]


In [11]:
def analyticsoln1(array):
        returnarray=np.exp(array)
        return returnarray
analyticxvals=analyticsoln1(tvals)
print(analyticxvals)

[1.01005017 1.02020134 1.03045453 1.04081077 1.0512711  1.06183655
 1.07250818 1.08328707 1.09417428]


# Implement RK4

In [12]:
def RK4(h,t,x,y,z,f,printdebug): #not a finite difference so no step in y
    k1= h*f(t,x,y,z)
    k2=h*f(t+h/2.,x+k1/2.,y,z)
    k3=h*f(t+h/2.,x+k2/2.,y,z)
    k4=h*f(t+h,x+k3,y,z)
    if(printdebug):
        print(f(t,x,y,z))
        print(t,x,y,z,h) 
        print("k1",k1)
        print("k2",k2)
        print("k3",k3)
        print("k4",k4)
        print(1./6.*(k1+2.*k2+2.*k3+k4))
        print(x+1/6.*(k1+2.*k2+2.*k3+k4))
    return t+h, x+1/6.*(k1+2.*k2+2.*k3+k4)



In [13]:
def RK4nodebug(h,t,x,y,z,f): #not a finite difference so no step in y
    k1= h*f(t,x,y,z)
    #print("k1",k1)
    k2=h*f(t+h/2.,x+k1/2.,y,z)
    #print("k2",k2)
    k3=h*f(t+h/2.,x+k2/2.,y,z)
    #print("k3",k3)
    k4=h*f(t+h,x+k3,y,z)
    #print("k4",k4)
    return t+h, x+1/6.*(k1+2.*k2+2.*k3+k4)



# Verifying the RK4 routine numerically

First I verify the RK4 by examining the behavior for a differential equation where the right hand side is a fourth order polynomial. This should have a truncation error that is on the order of the fourth order polynomial itself

In [14]:
def polynomial4(t,x,a,b,c,d,e):
    p=a*t**4+b*t**3+c*t**2+d*t+e
    #print(pi)
    return p
    

def polymaker(t,x,y,z,a,b,c,d,e):
    def poly(t,x,y,z):
        return polynomial4(t,x,a,b,c,d,e)
    return poly
        
        

In [15]:
import numpy as np
t=0.
x=1.0
y=0.
z=0.0
print(t,x)
polyfn=polymaker(t,x,y,z,1.,0.,0.,0.,0.)
polyfn1=polymaker1(t,x,y,z,1,-1.0)

0.0 1.0


In [16]:
#import matplotlib.pyplot as plt
#plt.plot(t,polyfn(t,x))

In [17]:
rk4polyout=RK4(.01,t,x,y,z,polyfn1,True)
print(x)
print(outp+x)
print(rk4polyout)

-1.0
0.0 1.0 0.0 0.0 0.01
k1 -0.01
k2 -0.00995
k3 -0.00995025
k4 -0.0099004975
-0.00995016625
0.99004983375
1.0
2.0
(0.01, 0.99004983375)


In [18]:
dt=0.01
numsteps=10
intvals=[]
tvals=[]
for i in np.arange(1,numsteps):
    t,x=RK4(dt,t,x,y,z,polyfn1,True)
    #print(valueofintegral)
    intvals.append(x)
    tvals.append(t)
print(intvals)
print(tvals)

-1.0
0.0 1.0 0.0 0.0 0.01
k1 -0.01
k2 -0.00995
k3 -0.00995025
k4 -0.0099004975
-0.00995016625
0.99004983375
-0.99004983375
0.01 0.99004983375 0.0 0.0 0.01
k1 -0.0099004983375
k2 -0.009850995845812501
k3 -0.009851243358270938
k4 -0.009801985903917291
-0.00985116044159736
0.9801986733084026
-0.9801986733084026
0.02 0.9801986733084026 0.0 0.0 0.01
k1 -0.009801986733084026
k2 -0.009752976799418606
k3 -0.009753221849086932
k4 -0.009704454514593158
-0.009753139757448043
0.9704455335509545
-0.9704455335509545
0.03 0.9704455335509545 0.0 0.0 0.01
k1 -0.009704455335509546
k2 -0.009655933058831999
k3 -0.009656175670215386
k4 -0.009607893578807392
-0.009656094395401951
0.9607894391555526
-0.9607894391555526
0.04 0.9607894391555526 0.0 0.0 0.01
k1 -0.009607894391555526
k2 -0.009559854919597748
k3 -0.009560095116957538
k4 -0.009512293440385951
-0.009560014650842007
0.9512294245047106
-0.9512294245047106
0.05 0.9512294245047106 0.0 0.0 0.01
k1 -0.009512294245047107
k2 -0.00946473277382187
k3 -0.0094

RK4 is less precise than Euler for a 1st order polynomial RHS

RK4 is precise to 1 part in 10^3 and Euler is precise to 1 part in 10^6 for this equation. 

# Check precision for a fourth order polynomial of RK4 relative to Euler. RK4 should do better 

In [19]:
polyfn4=polymaker1(t,x,y,z,4,-1.0)

In [20]:
dt=0.01
numsteps=30
integralvals=[]
tvals=[]
t=1.
x=(3.)**(-1./3.)
print(x)
for i in np.arange(1,numsteps):
    t,x=RK4(dt,t,x,y,z,polyfn4,True)
    #print(valueofintegral)
    integralvals.append(x)
    tvals.append(t)
print(np.array(integralvals))
print(tvals)
print(len(integralvals),len(tvals))

0.6933612743506348
-0.231120424783545
1.0 0.6933612743506348 0.0 0.0 0.01
k1 -0.00231120424783545
k2 -0.0022958346968050985
k3 -0.0022959366507114584
k4 -0.0022807434751047346
-0.0022959150696622165
0.6910653592809726
-0.22807437600030617
1.01 0.6910653592809726 0.0 0.0 0.01
k1 -0.002280743760003062
k2 -0.002265726568170315
k3 -0.002265825203258446
k4 -0.0022509786648749128
-0.002265804327955916
0.6887995549530167
-0.2250978937754673
1.02 0.6887995549530167 0.0 0.0 0.01
k1 -0.002250978937754673
k2 -0.0022363026773626576
k3 -0.0022363981327168697
k4 -0.00222188704279087
-0.0022363779334507662
0.6865631770195659
-0.22218873042700318
1.03 0.6865631770195659 0.0 0.0 0.01
k1 -0.002221887304270032
k2 -0.0022075410247283976
k3 -0.0022076334323280346
k4 -0.002193447067092411
-0.002207613880912551
0.6843555631386534
-0.2193447317751573
1.04 0.6843555631386534 0.0 0.0 0.01
k1 -0.0021934473177515727
k2 -0.0021794205213487984
k3 -0.002179510006495144
k4 -0.0021656380836222533
-0.002179491076176951

In [21]:
x0=(3.)**(-1./3.)
x0

0.6933612743506348

The analyitic solution to the differential equation

dx/dt = -x^4 

is 

x= (3t)^(-1/3)

Below the output is shown. The results are plotted for scaling comparison. It doesn't look like it scales correctly. '

In [22]:
def ODEsolnanalytic(t):
    return (3.*t)**(-1./3.)

In [23]:
solnanalytic=[]
for i in np.arange(len(tvals)):
    solnanalytic.append(ODEsolnanalytic(tvals[i]))
print(np.array(solnanalytic))
print(len(solnanalytic))

[0.69106536 0.68879955 0.68656318 0.68435556 0.68217607 0.68002408
 0.67789899 0.67580022 0.6737272  0.67167939 0.66965624 0.66765726
 0.66568193 0.66372977 0.66180031 0.6598931  0.65800768 0.65614362
 0.65430051 0.65247794 0.65067551 0.64889282 0.64712952 0.64538523
 0.64365959 0.64195226 0.64026291 0.63859119 0.6369368 ]
29


In [24]:
dt=0.01
numsteps=30
xvals=[]
tvals=[]
for i in np.arange(1,numsteps):
    #t=0.+i*dt
    t,x=euler(dt,t,x,y,z,polyfn4,True)
    print(x)
    xvals.append(x)
    tvals.append(t)
print(xvals)
print(tvals)

0.01 1.2900000000000003 0.6369368049183575 -0.0016987409780349837 0.6352380639403225
0.6352380639403225
0.01 1.3000000000000003 0.6352380639403225 -0.0016808341379371133 0.6335572298023854
0.6335572298023854
0.01 1.3100000000000003 0.6335572298023854 -0.0016632557774400702 0.6318939740249453
0.6318939740249453
0.01 1.3200000000000003 0.6318939740249453 -0.0016459973424855106 0.6302479766824598
0.6302479766824598
0.01 1.3300000000000003 0.6302479766824598 -0.001629050566696136 0.6286189261157636
0.6286189261157636
0.01 1.3400000000000003 0.6286189261157636 -0.001612407459548534 0.6270065186562152
0.6270065186562152
0.01 1.3500000000000003 0.6270065186562152 -0.0015960602951194728 0.6254104583610957
0.6254104583610957
0.01 1.3600000000000003 0.6254104583610957 -0.0015800016013736567 0.623830456759722
0.623830456759722
0.01 1.3700000000000003 0.623830456759722 -0.0015642241499630072 0.622266232609759
0.622266232609759
0.01 1.3800000000000003 0.622266232609759 -0.0015487209465093541 0.6207

In [25]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
# create a new plot with default tools, using figure
p = figure(plot_width=400, plot_height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(tvals,xvals, size=15, line_color="orange",fill_color="orange", fill_alpha=0.5, legend="Euler")
p.circle(tvals,integralvals, size=15, line_color="blue", fill_color="blue", fill_alpha=0.5,legend="RK4")
p.circle(tvals,solnanalytic, size=15, line_color="green",fill_color="green", fill_alpha=0.5,legend="dx/dt=-x^4 exact")


p.circle(tvals,np.array(solnanalytic)-np.array(xvals),size=15, line_color="yellow",fill_color="yellow",fill_alpha=0.5, legend="abs diff Euler")
p.circle(tvals,np.array(solnanalytic)-np.array(integralvals),size=15, line_color="red", fill_color="red", fill_alpha=0.5,legend="abs diff RK4")

p.legend.location = "bottom_left"
p.legend.click_policy="hide"

show(p) # show the results

Loading BokehJS ...

This looks good to me. RK4 performs better on the 4th order RHS ODE than Euler does, they are both converging after initial transients, they both have small truncation error relative to their absolute value which improves with choice of time step, and the truncation error is worse for higher order functions. This seems very reasonable. 

# An equal mass system will be easier to obtain the necessary numerical precision than the Earth-Sun system. 

So start with two stellar mass objects in a binary. 

In [54]:
import math
def InitialDataEqualMass():
    phi=np.array([0,math.pi])
    orbitalangle=np.zeros(2)
    orbitalradius=100*np.ones(2)
    eccentricity=np.zeros(2)
    mass=np.ones(2)
    masssun=1.989*10**30
    lsun=3.828*10**26
    massearth=5.9722*10**24
    masses=mass #*masssun (natural units)
    return phi,orbitalangle,orbitalradius,eccentricity, masses
    

In [66]:
import random,numpy as np
initdateq=InitialDataEqualMass()

In [67]:
print(initdateq)

(array([0.        , 3.14159265]), array([0., 0.]), array([100., 100.]), array([0., 0.]), array([1., 1.]))


In [74]:
def getxyuveq(initdat): #using reduced mass cooordinate system
    phi,orbitangle,orbitalradius,eccentricity, masses=initdat
    #print(orbitalradius, phi, np.cos(phi), np.sin(phi))
    metersperAU=149597870700
    Gconstant=6.408*10**-11
    metersperAU=1
    Gconstant=1
    x0=orbitalradius*np.cos(phi)*metersperAU
    y0=orbitalradius*np.sin(phi)*metersperAU
    count=0
    for phi0 in phi:
        if phi0==0:
            print("zero")
            x0[count]=orbitalradius[count]
            y0[count]=0
        if phi0==math.pi:
            print("pi")
            x0[count]=-orbitalradius[count]
            y0[count]=0
        count+=1
    z0=np.zeros(2)
    print("x0", x0)
    print("y0", y0)
    
    #start at perihelion for both (eliptical, doesn't generalize to three body)
    #actually start with circular orbit
    ux0=np.zeros(2) #*149597870700
    #centrepital force balances gravitational force
    reducedmass=np.zeros(2)
    masstotal=np.sum(masses)
    reducedmass[0]=masstotal
    print(masses)
    for i in np.arange(1):
        j=(i+1)%2 #reverse masses
        reducedmass[i+1]=masses[i]*masses[j]/np.sum(masses)
    print(reducedmass)
    metersperAU=1 #natural units
    rphys=np.zeros(len(masses))
    for i in np.arange(2):
        rphys[i]=orbitalradius[i]*metersperAU*reducedmass[i]/masses[i]
    x0[0]=0.0
    x0phys=rphys*x0/orbitalradius
    x0phys[0]=0.0
    y0phys=rphys*y0/orbitalradius
    z0phys=np.zeros(2)
    r0=np.sqrt(x0**2+y0**2)
    print(rphys)
    #G=1
    Gconstant=1
    F=(Gconstant*masstotal**2/rphys**2)
    print(F)
    #centF=reducedmass*v**2/rphys
    #centF=accel
    v=np.sqrt(Gconstant*masstotal/rphys)
    print(v)
    print("r0", r0)
    ux0=-v*y0/orbitalradius[1]
    uy0=v*x0/orbitalradius[1]
        
    #ux0=-v*y0/r0
    #uy0=v*x0/r0 #initial data in y only
    #evolve in plane only
    #circular orbit
    uz0=np.zeros(2)
    
    a=Gconstant*masstotal/(rphys**2)
    ax0=-a*x0/orbitalradius[1]
    ay0=-a*y0/orbitalradius[1]
    az0=np.zeros(2)
    
    
    return masstotal, reducedmass,x0phys,y0phys,z0phys, ux0, uy0,uz0, ax0, ay0,az0

In [75]:
xyuvaeq=getxyuveq(initdateq)
print(xyuvaeq)#In natural units
print(xyuvaeq[2][0])
print(xyuvaeq[1][0]/xyuvaeq[1][1])
print(xyuvaeq)

zero
pi
x0 [ 100. -100.]
y0 [0. 0.]
[1. 1.]
[2.  0.5]
[200.  50.]
[0.0001 0.0016]
[0.1 0.2]
r0 [  0. 100.]
(2.0, array([2. , 0.5]), array([  0., -50.]), array([0., 0.]), array([0., 0.]), array([-0., -0.]), array([ 0. , -0.2]), array([0., 0.]), array([-0.    ,  0.0008]), array([-0., -0.]), array([0., 0.]))
0.0
4.0
(2.0, array([2. , 0.5]), array([  0., -50.]), array([0., 0.]), array([0., 0.]), array([-0., -0.]), array([ 0. , -0.2]), array([0., 0.]), array([-0.    ,  0.0008]), array([-0., -0.]), array([0., 0.]))


In [76]:
def timestep(step,t,dt,mtotal, reducedmass,xi,yi,zi, vxi, vyi, vzi, axi, ayi, azi):
    xii=np.zeros(np.size(xi))
    vxii=np.zeros(np.size(vxi))
    yii=np.zeros(np.size(yi))
    vyii=np.zeros(np.size(vyi))
    zii=np.zeros(np.size(vzi))
    vzii=np.zeros(np.size(vzi))
    rii=np.zeros(np.size(xi))
    axii=np.zeros(np.size(axi))
    ayii=np.zeros(np.size(ayi))
    azii=np.zeros(np.size(azi))
    
    #This is redundant in the center of mass system
    Gconstant=6.408*10**-11
    Gconstant=1
    for k in np.arange(len(rii)):
        for j in np.arange(1,len(rii)):
            if j==0:
                axii[j]=0.0
                ayii[j]=0.0
                azii[j]=0.0
            if j!=k:
                rreljk=np.abs((xi[j] - xi[k])**2+(yi[j]-yi[k])**2+(zi[j]-zi[k])**2)**(1./2.)
                axii[j]-=Gconstant*reducedmass[k]*(xi[j]  - xi[k])/rreljk**3
                ayii[j]-=Gconstant*reducedmass[k]*(yi[j]  - yi[k])/rreljk**3
                azii[j]-=Gconstant*reducedmass[k]*(zi[j]  - zi[k])/rreljk**3
    #print(xii)
    
    for m in np.arange(len(x)):
        #m represents choices of mass
        i=step
        
        xii[m] = xi[m] + dt*vxi[m]
        #print(xii)
        vxii[m] = vxi[m] + dt*axii[m]
        #print(vxii)
        yii[m]= yi[m] + dt*vyi[m]
        vyii[m] = vyi[m] + dt*ayii[m]
        zii[m]= zi[m] + dt*vzi[m]
        vzii[m] = vzi[m] + dt*azii[m]
        rii[m]=np.sqrt(xi[m]**2+yi[m]**2+zi[m]**2)
    
    
    
    return reducedmass, xii,yii,zii,vxii,vyii,vzii,axii,ayii,azii
                    

In [81]:
dt=1 #*31556926 #seconds per year
numsteps=2000
masstotal,mass0,x,y,z0,vx,vy,vz0,ax,ay,az0=xyuvaeq
xcoord1=[]
xcoord2=[]
ycoord1=[]
ycoord2=[]
for i in np.arange(1,numsteps):
    t=0.+i*numsteps*dt
    mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(i,t,dt,masstotal,mass0,x,y,z0,vx,vy,vz0,ax,ay,az0)
    print(x,y,vx,vy,ax,ay)
    #print(ay) #forces should be equal and opposite, but in reduced mass framework accelerations are also equal and opposite
    #accelerations should evolve from y to x with time in a sinusoidal manner even in reduced mass framework
    #print(ax)
    xcoord1.append(x[0])
    xcoord2.append(x[1])
    ycoord1.append(y[0])
    ycoord2.append(y[1])
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(2,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(3,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(4,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 

[  0. -50.] [ 0.  -0.2] [0.     0.0008] [ 0.  -0.2] [0.     0.0008] [0. 0.]
[  0.     -49.9992] [ 0.  -0.4] [0.         0.00159998] [ 0.        -0.1999968] [0.         0.00079998] [0.0000000e+00 3.1999232e-06]
[  0.         -49.99760002] [ 0.        -0.5999968] [0.         0.00239993] [ 0.        -0.1999904] [0.         0.00079995] [0.00000000e+00 6.39969281e-06]
[  0.         -49.99520009] [ 0.        -0.7999872] [0.         0.00319983] [ 0.        -0.1999808] [0.        0.0007999] [0.00000000e+00 9.59925762e-06]
[  0.         -49.99200026] [ 0.       -0.999968] [0.         0.00399968] [ 0.       -0.199968] [0.         0.00079985] [0.00000000e+00 1.27985664e-05]
[  0.         -49.98800058] [ 0.       -1.199936] [0.         0.00479946] [ 0.       -0.199952] [0.         0.00079978] [0.0000000e+00 1.5997568e-05]
[  0.         -49.98320112] [ 0.         -1.39988801] [0.         0.00559915] [ 0.         -0.19993281] [0.         0.00079969] [0.00000000e+00 1.91962112e-05]
[  0.         -49.

[  0.         -39.19609737] [  0.         -31.17450732] [0.         0.12442253] [ 0.        -0.1568535] [0.         0.00062607] [0.         0.00049387]
[  0.         -39.07167484] [  0.         -31.33136082] [0.         0.12504661] [ 0.         -0.15635715] [0.         0.00062407] [0.         0.00049635]
[  0.         -38.94662823] [  0.         -31.48771797] [0.         0.12566867] [ 0.         -0.15585832] [0.         0.00062207] [0.         0.00049883]
[  0.         -38.82095956] [  0.         -31.64357628] [0.         0.12628872] [ 0.         -0.15535702] [0.         0.00062005] [0.        0.0005013]
[  0.         -38.69467084] [  0.        -31.7989333] [0.         0.12690674] [ 0.         -0.15485325] [0.         0.00061802] [0.         0.00050376]
[  0.        -38.5677641] [  0.         -31.95378655] [0.         0.12752273] [ 0.         -0.15434704] [0.         0.00061599] [0.         0.00050621]
[  0.         -38.44024137] [  0.         -32.10813359] [0.         0.12813668] [ 0.

[  0.         -14.07740765] [  0.         -48.21257578] [0.         0.19186244] [ 0.         -0.05691355] [0.         0.00022526] [0.         0.00076019]
[  0.         -13.88554521] [  0.         -48.26948933] [0.         0.19208465] [ 0.         -0.05615251] [0.         0.00022222] [0.         0.00076105]
[  0.         -13.69346056] [  0.         -48.32564184] [0.         0.19230382] [ 0.         -0.05539062] [0.         0.00021917] [0.         0.00076189]
[  0.         -13.50115674] [  0.         -48.38103246] [0.         0.19251995] [ 0.        -0.0546279] [0.         0.00021612] [0.         0.00076272]
[  0.         -13.30863679] [  0.         -48.43566036] [0.         0.19273302] [ 0.         -0.05386437] [0.         0.00021307] [0.         0.00076353]
[  0.         -13.11590378] [  0.         -48.48952473] [0.         0.19294303] [ 0.         -0.05310003] [0.         0.00021002] [0.         0.00076434]
[  0.         -12.92296074] [  0.         -48.54262477] [0.         0.19314999

[ 0.         18.49156375] [  0.         -46.98993283] [0.         0.18629792] [0.        0.0715693] [ 0.         -0.00028434] [0.         0.00073103]
[ 0.         18.67786168] [  0.         -46.91836353] [0.         0.18601072] [0.         0.07229914] [ 0.         -0.00028721] [0.         0.00072984]
[ 0.         18.86387239] [  0.         -46.84606439] [0.         0.18572065] [0.         0.07302778] [ 0.         -0.00029007] [0.         0.00072864]
[ 0.         19.04959304] [  0.         -46.77303662] [0.         0.18542773] [0.         0.07375521] [ 0.         -0.00029292] [0.         0.00072743]
[ 0.         19.23502077] [  0.         -46.69928141] [0.         0.18513196] [0.         0.07448141] [ 0.         -0.00029577] [0.         0.00072621]
[ 0.         19.42015274] [  0.     -46.6248] [0.         0.18483335] [0.         0.07520639] [ 0.         -0.00029861] [0.         0.00072498]
[ 0.         19.60498609] [  0.         -46.54959361] [0.        0.1845319] [0.         0.07593012

[ 0.         37.36992609] [  0.         -34.34492689] [0.         0.13610786] [0.         0.14505849] [ 0.         -0.00056962] [0.         0.00052764]
[ 0.         37.50603394] [  0.        -34.1998684] [0.         0.13553623] [0.         0.14558385] [ 0.         -0.00057163] [0.         0.00052535]
[ 0.         37.64157018] [  0.         -34.05428455] [0.         0.13496261] [0.         0.14610691] [ 0.         -0.00057362] [0.         0.00052306]
[ 0.         37.77653279] [  0.         -33.90817764] [0.         0.13438699] [0.         0.14662767] [ 0.         -0.00057561] [0.         0.00052076]
[ 0.         37.91091978] [  0.         -33.76154997] [0.        0.1338094] [0.         0.14714611] [ 0.         -0.00057759] [0.         0.00051845]
[ 0.         38.04472918] [  0.         -33.61440386] [0.         0.13322983] [0.         0.14766225] [ 0.         -0.00057957] [0.         0.00051613]
[ 0.         38.17795901] [  0.         -33.46674161] [0.         0.13264831] [0.         0.

...

[  0.         -24.60719578] [ 0.         46.20354458] [ 0.         -0.17229284] [ 0.         -0.09117086] [0.         0.00034069] [ 0.         -0.00064548]
[  0.         -24.77948862] [ 0.         46.11237372] [ 0.         -0.17194976] [ 0.         -0.09181505] [0.         0.00034308] [ 0.         -0.00064419]
[  0.         -24.95143838] [ 0.         46.02055867] [ 0.         -0.17160429] [ 0.         -0.09245794] [0.         0.00034547] [ 0.         -0.00064288]
[  0.         -25.12304267] [ 0.         45.92810073] [ 0.         -0.17125644] [ 0.         -0.09309951] [0.         0.00034785] [ 0.         -0.00064157]
[  0.         -25.29429911] [ 0.         45.83500122] [ 0.         -0.17090622] [ 0.         -0.09373976] [0.         0.00035022] [ 0.         -0.00064025]
[  0.         -25.46520533] [ 0.         45.74126147] [ 0.         -0.17055363] [ 0.         -0.09437868] [0.         0.00035259] [ 0.         -0.00063892]
[  0.         -25.63575896] [ 0.         45.64688279] [ 0.      

[  0.        -45.2231535] [ 0.        26.5484785] [ 0.         -0.09874982] [ 0.         -0.16812973] [0.         0.00062583] [ 0.         -0.00037053]
[  0.         -45.32190332] [ 0.         26.38034876] [ 0.         -0.09812262] [ 0.         -0.16849793] [0.         0.00062719] [ 0.        -0.0003682]
[  0.         -45.42002594] [ 0.         26.21185084] [ 0.         -0.09749407] [ 0.         -0.16886379] [0.         0.00062855] [ 0.         -0.00036586]
[  0.         -45.51752001] [ 0.         26.04298705] [ 0.         -0.09686418] [ 0.        -0.1692273] [0.        0.0006299] [ 0.         -0.00036351]
[  0.         -45.61438419] [ 0.         25.87375975] [ 0.         -0.09623294] [ 0.         -0.16958846] [0.         0.00063123] [ 0.         -0.00036116]
[  0.         -45.71061713] [ 0.         25.70417129] [ 0.         -0.09560038] [ 0.         -0.16994727] [0.         0.00063256] [ 0.         -0.00035881]
[  0.         -45.80621751] [ 0.         25.53422402] [ 0.        -0.09496

[  0.        -52.1538836] [0.         5.79507894] [ 0.         -0.02143536] [ 0.         -0.19398007] [0.         0.00072158] [ 0.00000000e+00 -8.28962535e-05]
[  0.         -52.17531895] [0.         5.60109887] [ 0.         -0.02071348] [ 0.         -0.19406028] [0.         0.00072188] [ 0.00000000e+00 -8.02113055e-05]
[  0.         -52.19603244] [0.         5.40703858] [ 0.         -0.01999132] [ 0.         -0.19413781] [0.         0.00072216] [ 0.00000000e+00 -7.75252877e-05]
[  0.         -52.21602375] [0.         5.21290077] [ 0.         -0.01926888] [ 0.         -0.19421265] [0.         0.00072244] [ 0.0000000e+00 -7.4838237e-05]
[  0.         -52.23529263] [0.         5.01868812] [ 0.         -0.01854617] [ 0.        -0.1942848] [0.         0.00072271] [ 0.00000000e+00 -7.21501903e-05]
[  0.         -52.25383881] [0.         4.82440333] [ 0.         -0.01782321] [ 0.         -0.19435426] [0.         0.00072296] [ 0.00000000e+00 -6.94611846e-05]
[  0.         -52.27166201] [0.   

[  0.         -50.18010911] [  0.         -15.44248784] [0.         0.05752783] [ 0.         -0.18669256] [0.         0.00069426] [0.         0.00021083]
[  0.         -50.12258128] [  0.         -15.62918041] [0.         0.05822129] [ 0.         -0.18647916] [0.         0.00069346] [0.         0.00021341]
[  0.         -50.06435999] [  0.         -15.81565956] [0.         0.05891394] [ 0.         -0.18626318] [0.         0.00069265] [0.         0.00021598]
[  0.         -50.00544605] [  0.         -16.00192274] [0.         0.05960578] [ 0.         -0.18604462] [0.         0.00069184] [0.         0.00021856]
[  0.         -49.94584027] [  0.         -16.18796736] [0.         0.06029679] [ 0.         -0.18582349] [0.         0.00069101] [0.         0.00022113]
[  0.         -49.88554348] [  0.         -16.37379085] [0.         0.06098697] [ 0.        -0.1855998] [0.         0.00069018] [0.         0.00022369]
[  0.         -49.82455651] [  0.         -16.55939065] [0.        0.0616763] 

[  0.         -27.68591481] [  0.         -44.72533168] [0.        0.1660002] [ 0.         -0.10328503] [0.         0.00038275] [0.         0.00061321]
[  0.        -27.5199146] [  0.         -44.82861671] [0.         0.16638066] [ 0.         -0.10267042] [0.         0.00038046] [0.         0.00061461]
[  0.         -27.35353394] [  0.         -44.93128713] [0.         0.16675882] [ 0.         -0.10205442] [0.         0.00037816] [0.       0.000616]
[  0.         -27.18677512] [  0.         -45.03334155] [0.         0.16713468] [ 0.         -0.10143703] [0.         0.00037586] [0.         0.00061739]
[  0.         -27.01964044] [  0.         -45.13477858] [0.         0.16750822] [ 0.         -0.10081827] [0.         0.00037355] [0.         0.00061876]
[  0.         -26.85213222] [  0.         -45.23559685] [0.         0.16787946] [ 0.         -0.10019815] [0.         0.00037123] [0.         0.00062012]
[  0.         -26.68425276] [  0.       -45.335795] [0.         0.16824837] [ 0.    

In [83]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
# create a new plot with default tools, using figure
p = figure(title="R0=100, Circular, Euler Method, Reduced Mass System", plot_width=400, plot_height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(xcoord1,ycoord1, size=15, line_color="purple",fill_color="purple", fill_alpha=0.5, legend="M")
p.circle(xcoord2,ycoord2, size=15, line_color="blue", fill_color="blue", fill_alpha=0.5,legend="mu")

p.legend.location = "bottom_left"
p.legend.click_policy="hide"

show(p) # show the results

Loading BokehJS ...

We have an orbit! The orbit doesn't yet conserve energy and angular momentum. In this case the inspiral or outspiral is not physical, unlike the General Relativistic case of gravitational waves. It is due to the problem conserving the conserved quantity. It is possible to enforce such conservation laws at each timestep. Implementing this might be the next step. Alternatively trying with a higher order differential equation solver might help, like the RK4. 

# First, compare to the two body form without the reduced mass

In [105]:
def getxyuveqnocm(initdat):
    phi,orbitangle,orbitalradius,eccentricity, masses=initdat
    #print(orbitalradius, phi, np.cos(phi), np.sin(phi))
    metersperAU=149597870700
    Gconstant=6.408*10**-11
    metersperAU=1
    Gconstant=1
    #fix x0 y0 at one star, disregard initial data, use orbital radius as separation between stars
    #this is consistant with choice in previous part
    x0=orbitalradius/2.*np.cos(phi)*metersperAU
    y0=orbitalradius/2.*np.sin(phi)*metersperAU
    count=0
    for phi0 in phi:
        if phi0==0:
            print("zero")
            x0[count]=orbitalradius[count]/2.
            y0[count]=0
        if phi0==math.pi:
            print("pi")
            x0[count]=-orbitalradius[count]/2.
            y0[count]=0
        count+=1
    #x0[1]=0.0
    #y0[1]=0.0
    z0=np.zeros(2)
    print(x0)
    print(y0)
    
    #start at perihelion for both (eliptical, doesn't generalize to three body)
    #actually start with circular orbit
    ux0=np.zeros(2) #*149597870700
    #centrepital force balances gravitational force
    metersperAU=1 #natural units
    #G=1
    Gconstant=1
    #Fcentripital=mass1*v**2/rphys
    #centF=accel
    #Faccel=G*m1*m2/r^2
    v=np.zeros(2)
    for i in np.arange(2):
        v[i]=np.sqrt(Gconstant*masses[(i+1)%2]/np.abs(orbitalradius[0]))
    print(v)
    r0=np.sqrt(x0**2+y0**2)
    ux0=-v*y0/r0
    uy0=v*x0/r0 #initial data in y only 
    uz0=np.zeros(2)
    
    a=np.zeros(2)
    for i in np.arange(2):
        a[i]=Gconstant*masses[(i+1)%2]/(orbitalradius[i]**2)
    ax0=-a*x0/r0
    ay0=-a*y0/r0
    az0=np.zeros(2)
    
    
    return masses,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0

In [106]:
xyuvaeqnocm=getxyuveqnocm(initdateq)
print(xyuvaeqnocm)#In natural units
print(xyuvaeqnocm[1][0])
print(xyuvaeqnocm[0][0]/xyuvaeq[1][1])
print(xyuvaeqnocm)

zero
pi
[ 0.5 -0.5]
[0. 0.]
[1. 1.]
(array([1., 1.]), array([ 0.5, -0.5]), array([0., 0.]), array([0., 0.]), array([-0., -0.]), array([ 1., -1.]), array([0., 0.]), array([-1.,  1.]), array([-0., -0.]), array([0., 0.]))
0.5
2.0
(array([1., 1.]), array([ 0.5, -0.5]), array([0., 0.]), array([0., 0.]), array([-0., -0.]), array([ 1., -1.]), array([0., 0.]), array([-1.,  1.]), array([-0., -0.]), array([0., 0.]))


In [176]:
def timestep(step,t,dt,mtotal, mass,xi,yi,zi, vxi, vyi, vzi, axi, ayi, azi):
    xii=np.zeros(np.size(xi))
    vxii=np.zeros(np.size(vxi))
    yii=np.zeros(np.size(yi))
    vyii=np.zeros(np.size(vyi))
    zii=np.zeros(np.size(vzi))
    vzii=np.zeros(np.size(vzi))
    rii=np.zeros(np.size(xi))
    axii=np.zeros(np.size(axi))
    ayii=np.zeros(np.size(ayi))
    azii=np.zeros(np.size(azi))
    
    #This is redundant in the center of mass system
    Gconstant=6.408*10**-11
    Gconstant=1
    for k in np.arange(len(rii)):
        for j in np.arange(len(rii)):
            if j!=k:
                rreljk=np.abs((xi[j] - xi[k])**2+(yi[j]-yi[k])**2+(zi[j]-zi[k])**2)**(1./2.)
                print(rreljk)
                axii[j]-=Gconstant*mass[k]*(xi[j]  - xi[k])/rreljk**3
                ayii[j]-=Gconstant*mass[k]*(yi[j]  - yi[k])/rreljk**3
                azii[j]-=Gconstant*mass[k]*(zi[j]  - zi[k])/rreljk**3
    #print(xii)
    
    for m in np.arange(len(x)):
        #m represents choices of mass
        i=step
        
        xii[m] = xi[m] + dt*vxi[m]
        vxii[m] = vxi[m] + dt*axi[m]
        yii[m]= yi[m] + dt*vyi[m]
        vyii[m] = vyi[m] + dt*ayi[m]
        zii[m]= zi[m] + dt*vzi[m]
        vzii[m] = vzi[m] + dt*azi[m]
        rii[m]=np.sqrt(xi[m]**2+yi[m]**2+zi[m]**2)
        print(vyii[m],vyi[m],ayi[m],dt)
        print(yii[m],yi[m],vyi[m],dt)
    
    
    return mass, xii,yii,zii,vxii,vyii,vzii,axii,ayii,azii
                    

In [177]:
dt=0.01 #*31556926 #seconds per year
numsteps=300
mass0,x,y,z0,vx,vy,vz0,ax,ay,az0=xyuvaeqnocm
xcoord1=[]
xcoord2=[]
ycoord1=[]
ycoord2=[]
for i in np.arange(1,numsteps):
    mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(i,t,dt,masstotal,mass0,x,y,z0,vx,vy,vz0,ax,ay,az0)
    print(x,y,vx,vy,ax,ay)
    #print(ay) #forces should be equal and opposite, but in reduced mass framework accelerations are also equal and opposite
    #accelerations should evolve from y to x with time in a sinusoidal manner even in reduced mass framework
    #print(ax)
    xcoord1.append(x[0])
    xcoord2.append(x[1])
    ycoord1.append(y[0])
    ycoord2.append(y[1])
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(2,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(3,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(4,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 

1.0
1.0
1.0 1.0 -0.0 0.01
0.01 0.0 1.0 0.01
-1.0 -1.0 -0.0 0.01
-0.01 0.0 -1.0 0.01
[ 0.5 -0.5] [ 0.01 -0.01] [-0.01  0.01] [ 1. -1.] [-1.  1.] [0. 0.]
1.000199980003999
1.000199980003999
1.0 1.0 0.0 0.01
0.02 0.01 1.0 0.01
-1.0 -1.0 0.0 0.01
-0.02 -0.01 -1.0 0.01
[ 0.4999 -0.4999] [ 0.02 -0.02] [-0.02  0.02] [ 1. -1.] [-0.9994003  0.9994003] [-0.01998801  0.01998801]
1.0005998400959297
1.0005998400959297
0.999800119940028 1.0 -0.019988005997201266 0.01
0.03 0.02 1.0 0.01
-0.999800119940028 -1.0 0.019988005997201266 0.01
-0.03 -0.02 -1.0 0.01
[ 0.4997 -0.4997] [ 0.03 -0.03] [-0.029994  0.029994] [ 0.99980012 -0.99980012] [-0.998003  0.998003] [-0.03992811  0.03992811]
1.0011994606470782
1.0011994606470782
0.9994008388854662 0.999800119940028 -0.03992810545618887 0.01
0.03999800119940028 0.03 0.999800119940028 0.01
-0.9994008388854662 -0.999800119940028 0.03992810545618887 0.01
-0.03999800119940028 -0.03 -0.999800119940028 0.01
[ 0.49940006 -0.49940006] [ 0.039998 -0.039998] [-0.0399740

0.7169504973883896 0.7215289680223722 -0.45784706339826853 0.01
0.647985653913986 0.6407703642337623 0.7215289680223722 0.01
-0.7169504973883896 -0.7215289680223722 0.45784706339826853 0.01
-0.647985653913986 -0.6407703642337623 -0.7215289680223722 0.01
[ 0.29319429 -0.29319429] [ 0.64798565 -0.64798565] [-0.46320292  0.46320292] [ 0.7169505 -0.7169505] [-0.21103893  0.21103893] [-0.45408149  0.45408149]
1.422460258382705
1.422460258382705
0.7124096825139189 0.7169504973883896 -0.4540814874470686 0.01
0.6551551588878699 0.647985653913986 0.7169504973883896 0.01
-0.7124096825139189 -0.7169504973883896 0.4540814874470686 0.01
-0.6551551588878699 -0.647985653913986 -0.7169504973883896 0.01
[ 0.28856226 -0.28856226] [ 0.65515516 -0.65515516] [-0.46531331  0.46531331] [ 0.71240968 -0.71240968] [-0.20373473  0.20373473] [-0.45027202  0.45027202]
1.4317771585099235
1.4317771585099235
0.7079069623033736 0.7124096825139189 -0.4502720210545279 0.01
0.662279255713009 0.6551551588878699 0.71240968

-1.0210372052434271 -1.0159594556117812 -0.5077749631645785 0.01
[-0.01479679  0.01479679] [ 1.02103721 -1.02103721] [-0.50980609  0.50980609] [ 0.50532845 -0.50532845] [ 0.00231186 -0.00231186] [-0.24217421  0.24217421]
2.042288833049815
2.042288833049815
0.502906710594427 0.5053284526926963 -0.24217420982693033 0.01
1.026090489770354 1.0210372052434271 0.5053284526926963 0.01
-0.502906710594427 -0.5053284526926963 0.24217420982693033 0.01
-1.026090489770354 -1.0210372052434271 -0.5053284526926963 0.01
[-0.01989485  0.01989485] [ 1.02609049 -1.02609049] [-0.50978297  0.50978297] [ 0.50290671 -0.50290671] [ 0.00347413 -0.00347413] [-0.23972872  0.23972872]
2.0525666841045265
2.0525666841045265
0.500509423347255 0.502906710594427 -0.23972872471720558 0.01
1.0311195568762983 1.026090489770354 0.502906710594427 0.01
-0.500509423347255 -0.502906710594427 0.23972872471720558 0.01
-1.0311195568762983 -1.026090489770354 -0.502906710594427 0.01
[-0.02499268  0.02499268] [ 1.03111956 -1.0311195

0.39020517421520234 0.3915723187916464 -0.13671445764440507 0.01
1.3002000945824792 1.2962843713945627 0.3915723187916464 0.01
-0.39020517421520234 -0.3915723187916464 0.13671445764440507 0.01
-1.3002000945824792 -1.2962843713945627 -0.3915723187916464 0.01
[-0.33244785  0.33244785] [ 1.30020009 -1.30020009] [-0.4951647  0.4951647] [ 0.39020517 -0.39020517] [ 0.03425583 -0.03425583] [-0.13559169  0.13559169]
2.6840580157720413
2.6840580157720413
0.38884925735081743 0.39020517421520234 -0.1355916864384928 0.01
1.3041021463246312 1.3002000945824792 0.39020517421520234 0.01
-0.38884925735081743 -0.39020517421520234 0.1355916864384928 0.01
-1.3041021463246312 -1.3002000945824792 -0.39020517421520234 0.01
[-0.3373995  0.3373995] [ 1.30410215 -1.30410215] [-0.49482214  0.49482214] [ 0.38884926 -0.38884926] [ 0.0343857 -0.0343857] [-0.1344821  0.1344821]
2.6940830185429823
2.6940830185429823
0.38750443631918585 0.38884925735081743 -0.13448210316316087 0.01
1.3079906388981393 1.304102146324631

-1.5200628482520226 -1.5168234249031374 -0.3239423348885293 0.01
[-0.63272534  0.63272534] [ 1.52006285 -1.52006285] [-0.4730102  0.4730102] [ 0.32307956 -0.32307956] [ 0.03548339 -0.03548339] [-0.08570501  0.08570501]
3.292981878334351
3.292981878334351
0.32222250823932663 0.3230795583569094 -0.08570501175827622 0.01
1.5232936438355917 1.5200628482520226 0.3230795583569094 0.01
-0.32222250823932663 -0.3230795583569094 0.08570501175827622 0.01
-1.5232936438355917 -1.5200628482520226 -0.3230795583569094 0.01
[-0.63745544  0.63745544] [ 1.52329364 -1.52329364] [-0.47265537  0.47265537] [ 0.32222251 -0.32222251] [ 0.03543865 -0.03543865] [-0.08513801  0.08513801]
3.3025886574651824
3.3025886574651824
0.32137112809391033 0.32222250823932663 -0.08513801454162831 0.01
1.526515868917985 1.5232936438355917 0.32222250823932663 0.01
-0.32137112809391033 -0.32222250823932663 0.08513801454162831 0.01
-1.526515868917985 -1.5232936438355917 -0.32222250823932663 0.01
[-0.64218199  0.64218199] [ 1.526

In [178]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
# create a new plot with default tools, using figure
p = figure(title="No y acceleration, Euler Method, Equal Mass System", plot_width=400, plot_height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(xcoord1,ycoord1, size=15, line_color="purple",fill_color="purple", fill_alpha=0.5, legend="M")
p.circle(xcoord2,ycoord2, size=15, line_color="blue", fill_color="blue", fill_alpha=0.5,legend="mu")

p.legend.location = "bottom_left"
p.legend.click_policy="hide"

show(p) # show the results

Loading BokehJS ...

Unphysical drift outward found after a quarter cycle for an equal mass systerm without the center of mass frame (no reduced mass coordinates). It is because there is no y acceleration because the y initial conditions are on the x axis and the y acceleration is based on the difference between the y coordinates, which is zero, and the initial y acceleration is also zero. This could be improved by choosing a different initial condition where y is nonzero for both stars or by artificially conserving energy and momentum or fixing the radius for this circular problem. 

It doesn't improve with a 10 times smaller time step. 

# Try initial condition offset from y axis

In [184]:
import math
def InitialDataEqualMassOffset():
    phi=np.array([math.pi/6,7*math.pi/6])
    orbitalangle=np.zeros(2)
    orbitalradius=100*np.ones(2)
    eccentricity=np.zeros(2)
    mass=np.ones(2)
    masssun=1.989*10**30
    lsun=3.828*10**26
    massearth=5.9722*10**24
    masses=mass #*masssun (natural units)
    return phi,orbitalangle,orbitalradius,eccentricity, masses
    

In [185]:
import random,numpy as np
initdateqoffset=InitialDataEqualMassOffset()

In [186]:
xyuvaeqoffset=getxyuveqnocm(initdateqoffset)
print(xyuvaeqoffset)#In natural units
print(xyuvaeqoffset[1][0])
print(xyuvaeqoffset[0][0]/xyuvaeqoffset[1][1])
print(xyuvaeqoffset)

[ 43.30127019 -43.30127019]
[ 25. -25.]
[0.1 0.1]
(array([1., 1.]), array([ 43.30127019, -43.30127019]), array([ 25., -25.]), array([0., 0.]), array([-0.05,  0.05]), array([ 0.08660254, -0.08660254]), array([0., 0.]), array([-8.66025404e-05,  8.66025404e-05]), array([-5.e-05,  5.e-05]), array([0., 0.]))
43.30127018922194
-0.02309401076758503
(array([1., 1.]), array([ 43.30127019, -43.30127019]), array([ 25., -25.]), array([0., 0.]), array([-0.05,  0.05]), array([ 0.08660254, -0.08660254]), array([0., 0.]), array([-8.66025404e-05,  8.66025404e-05]), array([-5.e-05,  5.e-05]), array([0., 0.]))


In [200]:
dt=10. #*31556926 #seconds per year
numsteps=3000
mass0,x,y,z0,vx,vy,vz0,ax,ay,az0=xyuvaeqoffset
xcoord1=[]
xcoord2=[]
ycoord1=[]
ycoord2=[]
for i in np.arange(1,numsteps):
    mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(i,t,dt,masstotal,mass0,x,y,z0,vx,vy,vz0,ax,ay,az0)
    print(x,y,vx,vy,ax,ay)
    #print(ay) #forces should be equal and opposite, but in reduced mass framework accelerations are also equal and opposite
    #accelerations should evolve from y to x with time in a sinusoidal manner even in reduced mass framework
    #print(ax)
    xcoord1.append(x[0])
    xcoord2.append(x[1])
    ycoord1.append(y[0])
    ycoord2.append(y[1])
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(2,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(3,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(4,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 

100.0
100.0
0.08610254037844388 0.08660254037844388 -4.9999999999999996e-05 10.0
25.866025403784434 24.999999999999996 0.08660254037844388 10.0
-0.08610254037844388 -0.08660254037844388 4.999999999999998e-05 10.0
-25.866025403784427 -24.99999999999999 -0.08660254037844388 10.0
[ 42.80127019 -42.80127019] [ 25.8660254 -25.8660254] [-0.05086603  0.05086603] [ 0.08610254 -0.08610254] [-8.66025404e-05  8.66025404e-05] [-5.e-05  5.e-05]
100.0199980003999
100.0199980003999
0.08560254037844388 0.08610254037844388 -4.999999999999999e-05 10.0
26.727050807568872 25.866025403784434 0.08610254037844388 10.0
-0.08560254037844388 -0.08610254037844388 4.999999999999999e-05 10.0
-26.727050807568865 -25.866025403784427 -0.08610254037844388 10.0
[ 42.29260994 -42.29260994] [ 26.72705081 -26.72705081] [-0.05173205  0.05173205] [ 0.08560254 -0.08560254] [-8.55512045e-05  8.55512045e-05] [-5.17010271e-05  5.17010271e-05]
100.05998400959297
100.05998400959297
0.08508553010754928 0.08560254037844388 -5.17010

[-6.24789542  6.24789542] [ 70.38256436 -70.38256436] [-0.07600173  0.07600173] [ 0.03943537 -0.03943537] [ 3.96592577e-06 -3.96592577e-06] [-5.05778967e-05  5.05778967e-05]
141.318669146703
141.318669146703
0.038929588287384014 0.03943536725488385 -5.0577896749983556e-05 10.0
70.77691802773784 70.382564355189 0.03943536725488385 10.0
-0.038929588287384014 -0.03943536725488385 5.0577896749983556e-05 10.0
-70.77691802773784 -70.382564355189 -0.03943536725488385 10.0
[-7.00791276  7.00791276] [ 70.77691803 -70.77691803] [-0.07596207  0.07596207] [ 0.03892959 -0.03892959] [ 4.42756688e-06 -4.42756688e-06] [-4.98765569e-05  4.98765569e-05]
142.24602583827033
142.24602583827033
0.03843082271867734 0.038929588287384014 -4.9876556870667175e-05 10.0
71.16621391061167 70.77691802773784 0.038929588287384014 10.0
-0.03843082271867734 -0.038929588287384014 4.9876556870667175e-05 10.0
-71.16621391061167 -70.77691802773784 -0.038929588287384014 10.0
[-7.7675335  7.7675335] [ 71.16621391 -71.16621391

[-38.18977926  38.18977926] [ 83.54622795 -83.54622795] [-0.07189342  0.07189342] [ 0.02308669 -0.02308669] [ 1.22880479e-05 -1.22880479e-05] [-2.73221108e-05  2.73221108e-05]
183.721870720672
183.721870720672
0.02281347306788589 0.02308669417637693 -2.7322110849103822e-05 10.0
83.77709489523095 83.54622795346718 0.02308669417637693 10.0
-0.02281347306788589 -0.02308669417637693 2.7322110849103822e-05 10.0
-83.77709489523095 -83.54622795346718 -0.02308669417637693 10.0
[-38.90871341  38.90871341] [ 83.7770949 -83.7770949] [-0.07177053  0.07177053] [ 0.02281347 -0.02281347] [ 1.23167058e-05 -1.23167058e-05] [-2.69447566e-05  2.69447566e-05]
184.74295232757888
184.74295232757888
0.022544025501438882 0.02281347306788589 -2.6944756644700584e-05 10.0
84.0052296259098 83.77709489523095 0.02281347306788589 10.0
-0.022544025501438882 -0.02281347306788589 2.6944756644700584e-05 10.0
-84.0052296259098 -83.77709489523095 -0.02281347306788589 10.0
[-39.62641876  39.62641876] [ 84.00522963 -84.0052

93.94354341671067 93.82948649960757 0.011405691710309545 10.0
-0.011280094168128934 -0.011405691710309545 1.2559754218061113e-05 10.0
-93.94354341671067 -93.82948649960757 -0.011405691710309545 10.0
[-81.1106904  81.1106904] [ 93.94354342 -93.94354342] [-0.06443662  0.06443662] [ 0.01128009 -0.01128009] [ 1.06517702e-05 -1.06517702e-05] [-1.24208907e-05  1.24208907e-05]
248.22839037161285
248.22839037161285
0.011155885261144026 0.011280094168128934 -1.242089069849074e-05 10.0
94.05634435839195 93.94354341671067 0.011280094168128934 10.0
-0.011155885261144026 -0.011280094168128934 1.242089069849074e-05 10.0
-94.05634435839195 -93.94354341671067 -0.011280094168128934 10.0
[-81.75505663  81.75505663] [ 94.05634436 -94.05634436] [-0.06433011  0.06433011] [ 0.01115589 -0.01115589] [ 1.06060517e-05 -1.06060517e-05] [-1.22840784e-05  1.22840784e-05]
249.24273468441834
249.24273468441834
0.011033044477180521 0.011155885261144026 -1.2284078396350355e-05 10.0
94.16790321100339 94.05634435839195 

[-119.21553287  119.21553287] [ 99.01290324 -99.01290324] [-0.0586602  0.0586602] [ 0.00556598 -0.00556598] [ 8.0442321e-06 -8.0442321e-06] [-6.71029406e-06  6.71029406e-06]
309.94127370240074
309.94127370240074
0.005498875001807518 0.005565977942453092 -6.710294064557365e-06 10.0
99.0685630206572 99.01290324123266 0.005565977942453092 10.0
-0.005498875001807518 -0.005565977942453092 6.710294064557365e-06 10.0
-99.0685630206572 -99.01290324123266 -0.005565977942453092 10.0
[-119.80213491  119.80213491] [ 99.06856302 -99.06856302] [-0.05857976  0.05857976] [ 0.00549888 -0.00549888] [ 8.00800987e-06 -8.00800987e-06] [-6.65094797e-06  6.65094797e-06]
310.91562654781245
310.91562654781245
0.005432365522110358 0.005498875001807518 -6.6509479697159436e-06 10.0
99.12355177067526 99.0685630206572 0.005498875001807518 10.0
-0.005432365522110358 -0.005498875001807518 6.6509479697159436e-06 10.0
-99.12355177067526 -99.0685630206572 -0.005498875001807518 10.0
[-120.38793252  120.38793252] [ 99.123

...

1945.5872366045055
-0.005727612627041858 -0.005727654508505262 4.188146340448767e-09 10.0
-15.523897152861263 -15.46662060777621 -0.005727654508505262 10.0
0.005727612627041858 0.005727654508505262 -4.188146340448767e-09 10.0
15.523897152861263 15.46662060777621 0.005727654508505262 10.0
[-972.93838996  972.93838996] [-15.52389715  15.52389715] [-0.02677063  0.02677063] [-0.00572761  0.00572761] [ 2.64145785e-07 -2.64145785e-07] [ 4.20023222e-09 -4.20023222e-09]
1946.1244585408024
1946.1244585408024
-0.005727570624719614 -0.005727612627041858 4.20023222433717e-09 10.0
-15.581173279131681 -15.523897152861263 -0.005727612627041858 10.0
0.005727570624719614 0.005727612627041858 -4.20023222433717e-09 10.0
15.581173279131681 15.523897152861263 0.005727612627041858 10.0
[-973.20609624  973.20609624] [-15.58117328  15.58117328] [-0.02676799  0.02676799] [-0.00572757  0.00572757] [ 2.63999742e-07 -2.63999742e-07] [ 4.21229637e-09 -4.21229637e-09]
1946.6616333851491
1946.6616333851491
-0.005727

0.005724788340766379 0.005724837318953305 -4.897818692520385e-09 10.0
19.074178276400385 19.01692990321085 0.005724837318953305 10.0
[-989.48677475  989.48677475] [-19.07417828  19.07417828] [-0.02660957  0.02660957] [-0.00572479  0.00572479] [ 2.55336529e-07 -2.55336529e-07] [ 4.90862869e-09 -4.90862869e-09]
1979.341205230471
1979.341205230471
-0.005724739254479518 -0.005724788340766379 4.908628686141753e-09 10.0
-19.13142615980805 -19.074178276400385 -0.005724788340766379 10.0
0.005724739254479518 0.005724788340766379 -4.908628686141753e-09 10.0
19.13142615980805 19.074178276400385 0.005724788340766379 10.0
[-989.75287045  989.75287045] [-19.13142616  19.13142616] [-0.02660702  0.02660702] [-0.00572474  0.00572474] [ 2.55198427e-07 -2.55198427e-07] [ 4.91941925e-09 -4.91941925e-09]
1979.8755072372262
1979.8755072372262
-0.005724690060287058 -0.005724739254479518 4.919419246010581e-09 10.0
-19.188673552352846 -19.13142615980805 -0.005724739254479518 10.0
0.005724690060287058 0.0057247

2012.3825760044879
2012.3825760044879
-0.005721492666704266 -0.005721548089046047 5.542234178109309e-09 10.0
-22.679794557113247 -22.622579076222788 -0.005721548089046047 10.0
0.005721492666704266 0.005721548089046047 -5.542234178109309e-09 10.0
22.679794557113247 22.622579076222788 0.005721548089046047 10.0
[-1006.20147813  1006.20147813] [-22.67979456  22.67979456] [-0.02645136  0.02645136] [-0.00572149  0.00572149] [ 2.46870449e-07 -2.46870449e-07] [ 5.55188505e-09 -5.55188505e-09]
2012.9140942208564
2012.9140942208564
-0.005721437147853784 -0.005721492666704266 5.551885048231215e-09 10.0
-22.73700948378029 -22.679794557113247 -0.005721492666704266 10.0
0.005721437147853784 0.005721492666704266 -5.551885048231215e-09 10.0
22.73700948378029 22.679794557113247 0.005721492666704266 10.0
[-1006.46599171  1006.46599171] [-22.73700948  22.73700948] [-0.02644889  0.02644889] [-0.00572144  0.00572144] [ 2.46739809e-07 -2.46739809e-07] [ 5.56151854e-09 -5.56151854e-09]
2013.4455682418904
201

0.005717874721585952 0.005717935810155244 -6.108856929146195e-09 10.0
26.226034663264112 26.16885530516256 0.005717935810155244 10.0
[-1022.5551245  1022.5551245] [-26.22603466  26.22603466] [-0.02630073  0.02630073] [-0.00571787  0.00571787] [ 2.38980946e-07 -2.38980946e-07] [ 6.11748621e-09 -6.11748621e-09]
2045.7827719731156
2045.7827719731156
-0.005717813546723806 -0.005717874721585952 6.117486214521651e-09 10.0
-26.283213410479974 -26.226034663264112 -0.005717874721585952 10.0
0.005717813546723806 0.005717874721585952 -6.117486214521651e-09 10.0
26.283213410479974 26.226034663264112 0.005717874721585952 10.0
[-1022.81813176  1022.81813176] [-26.28321341  26.28321341] [-0.02629834  0.02629834] [-0.00571781  0.00571781] [ 2.3885711e-07 -2.3885711e-07] [ 6.12609991e-09 -6.12609991e-09]
2046.311548084446
2046.311548084446
-0.0057177522857246805 -0.005717813546723806 6.126099912576198e-09 10.0
-26.340391545947213 -26.283213410479974 -0.005717813546723806 10.0
0.0057177522857246805 0.00

[-1038.8168459  1038.8168459] [-29.76992845  29.76992845] [-0.02615486  0.02615486] [-0.00571392  0.00571392] [ 2.31498286e-07 -2.31498286e-07] [ 6.62310402e-09 -6.62310402e-09]
2078.4866494281982
2078.4866494281982
-0.005713858501593545 -0.005713924732633787 6.6231040242597086e-09 10.0
-29.8270676951825 -29.769928447856163 -0.005713924732633787 10.0
0.005713858501593545 0.005713924732633787 -6.6231040242597086e-09 10.0
29.8270676951825 29.769928447856163 0.005713924732633787 10.0
[-1039.0783945  1039.0783945] [-29.8270677  29.8270677] [-0.02615255  0.02615255] [-0.00571386  0.00571386] [ 2.3138077e-07 -2.3138077e-07] [ 6.63080213e-09 -6.63080213e-09]
2079.0128079342544
2079.0128079342544
-0.005713792193572195 -0.005713858501593545 6.6308021349641665e-09 10.0
-29.884206280198434 -29.8270676951825 -0.005713858501593545 10.0
0.005713792193572195 0.005713858501593545 -6.6308021349641665e-09 10.0
29.884206280198434 29.8270676951825 0.005713858501593545 10.0
[-1039.33991996  1039.33991996] 

In [201]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
# create a new plot with default tools, using figure
p = figure(title="30 degree init, Euler Method, Equal Mass System", plot_width=400, plot_height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(xcoord1,ycoord1, size=15, line_color="purple",fill_color="purple", fill_alpha=0.5, legend="M")
p.circle(xcoord2,ycoord2, size=15, line_color="blue", fill_color="blue", fill_alpha=0.5,legend="mu")

p.legend.location = "bottom_left"
p.legend.click_policy="hide"

show(p) # show the results

Loading BokehJS ...

This is a hyperbolic or parabolic trajectory where the center of mass isn't the focus. This is really weird. The initial condition has too much velocity for its acceleration, I think. 

# Two body circular orbit equal mass again, using RK4

In [50]:
def RK4implicit(h,t,x,f): #not a finite difference so no step in y
    k1= h*f(t)
    k2=h*f(t+h/2.)
    k3=h*f(t+h/2.)
    k4=h*f(t+h)
    return t+h, x+1/6.*(k1+2.*k2+2.*k3+k4)




In [92]:
class OrbitDiffEq:
    def __init__(self,reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0,t0):
        self.reducedmass=reducedmass
        self.xi=x0
        self.yi=y0
        self.zi=z0
        self.vxi=ux0
        self.vyi=uy0
        self.vzi=uz0
        self.axi=ax0
        self.ayi=ay0
        self.azi=az0
        self.ti=t0
    def dxidt(self,t):
        return self.vxi
    def dyidt(self,t):
        return self.vyi
    def dzidt(self,t):
        return self.vzi
    def dvxidt(self,t):
        #return axi[m]
        axii=np.zeros(np.shape(self.axi))
        rii=np.sqrt(self.xi**2+self.yi**2+self.zi**2)
        Gconstant=6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(1,len(rii)):
                if j==0:
                    axii[j]=0.0
                if j!=k:
                    rreljk=np.abs((self.xi[j] - self.xi[k])**2+(self.yi[j]-self.yi[k])**2+(self.zi[j]-self.zi[k])**2)**(1./2.)
                    axii[j]+=Gconstant*self.reducedmass[k]*(self.xi[j]  - self.xi[k])/rreljk**3
        self.axi=axii
        return axii
    def dvyidt(self,t):
        #return axi[m]
        ayii=np.zeros(np.shape(self.ayi))
        rii=np.sqrt(self.xi**2+self.yi**2+self.zi**2)
        Gconstant=6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(1,len(rii)):
                if j==0:
                    ayii[j]=0.0
                if j!=k:
                    rreljk=np.abs((self.xi[j] - self.xi[k])**2+(self.yi[j]-self.yi[k])**2+(self.zi[j]-self.zi[k])**2)**(1./2.)
                    ayii[j]+=Gconstant*self.reducedmass[k]*(self.yi[j]  - self.yi[k])/rreljk**3
        self.ayi=ayii
        return ayii
    def dvzidt(self,t):
        #return axi[m]
        azii=np.zeros(np.shape(self.azi))
        rii=np.sqrt(self.xi**2+self.yi**2+self.zi**2)
        Gconstant=6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(1,len(rii)):
                if j==0:
                    azii[j]=0.0
                if j!=k:
                    rreljk=np.abs((self.xi[j] - self.xi[k])**2+(self.yi[j]-self.yi[k])**2+(self.zi[j]-self.zi[k])**2)**(1./2.)
                    azii[j]+=Gconstant*self.reducedmass[k]*(self.zi[j]  - self.zi[k])/rreljk**3
        self.azi=azii
        return azii
    def updateINTERNAL(self,xii,yii,zii,vxii,vyii,vzii,tii):
        self.xi=xii
        self.yi=yii
        self.zi=zii
        self.vxi=vxii
        self.vyi=vyii
        self.vzi=vzii
        self.ti=tii
        return self
    def update(self,xii,yii,zii,vxii,vyii,vzii,axii,ayii,azii,tii):
        self.xi=xii
        self.yi=yii
        self.zi=zii
        self.vxi=vxii
        self.vyi=vyii
        self.vzi=vzii
        self.axi=axii
        self.ayi=ayii
        self.azi=azii
        self.ti=tii
    def print2D(self):
        print(self.reducedmass,self.xi,self.yi,self.vxi,self.vyi,self.axi,self.ayi,self.ti)
        return self
    def list2D(self):
        return self.reducedmass,self.xi,self.yi,self.vxi,self.vyi,self.axi,self.ayi,self.ti
    def timestepRK4ODE(self,step,dt):

    
        h=dt
        #tnew,ynew, intval=RK4(h,t,y,f)
        #m represents choices of mass
        i=step
        
        tnew,intvalx=RK4implicit(h,self.ti,self.dxidt)
        xii = intvalx
        tnew,intvalvx=RK4implicit(h,self.ti,self.dvxidt)
        vxii=intvalvx
        tnew,intvaly=RK4implicit(h,self.ti,self.dyidt)
        yii = intvaly
        tnew,intvalvy=RK4implicit(h,self.ti,self.dvyidt)
        vyii=intvalvy
        tnew,intvalz=RK4implicit(h,self.ti,self.dzidt)
        zii = intvalz
        tnew,intvalvz=RK4implicit(h,self.ti,self.dvzidt)
        vzii=intvalvz
 
        #print(xii)
        self.updateINTERNAL(xii,yii,zii,vxii,vyii,vzii,tnew)
        return reducedmass, xii,yii,zii,vxii,vyii,vzii,self.axi,self.ayi,self.azi,self.ti

In [93]:
print(xyuvaeq)

(2.0, array([2. , 0.5]), array([ 0. , -0.5]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([ 0., -2.]), array([0., 0.]), array([-0.,  8.]), array([-0., -0.]), array([0., 0.]))


In [99]:
mtotal,reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0=xyuvaeq
ODEeq= OrbitDiffEq(reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0,0.0)
ODEeq.print2D()
ODEeq.print2D()

[2.  0.5] [ 0.  -0.5] [0. 0.] [0. 0.] [ 0. -2.] [-0.  8.] [-0. -0.] 0.0
[2.  0.5] [ 0.  -0.5] [0. 0.] [0. 0.] [ 0. -2.] [-0.  8.] [-0. -0.] 0.0


In [95]:
mtotal,reducedmass,x,y,z,vx,vy,vz,ax,ay,az=xyuvaeq
print(x,y,vx,vy,ax,ay)
print(xyuvaeq)

[ 0.  -0.5] [0. 0.] [0. 0.] [ 0. -2.] [-0.  8.] [-0. -0.]
(2.0, array([2. , 0.5]), array([ 0. , -0.5]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([ 0., -2.]), array([0., 0.]), array([-0.,  8.]), array([-0., -0.]), array([0., 0.]))


In [102]:
dt=0.001 #natural units #*31556926 #seconds per year
numsteps=5
mtotal,reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0=xyuvaeq
ODEeq= OrbitDiffEq(reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0,0.0)
mtotal,reducedmass,x,y,z,vx,vy,vz,ax,ay,az=xyuvaeq
ODEeq.print2D()
t=0.0
for i in np.arange(1,numsteps):
    reducedmass,x,y,z,vx,vy,vz,ax,ay,az,t=ODEeq.timestepRK4ODE(i,dt)
    ODEeq.print2D()
    ODEeq.update(x,y,z,vx,vy,vz,ax,ay,az,t)



[2.  0.5] [ 0.  -0.5] [0. 0.] [0. 0.] [ 0. -2.] [-0.  8.] [-0. -0.] 0.0
[0. 0.] [ 0.    -2.002] [0. 0.]
[2.  0.5] [0. 0.] [ 0.    -2.002] [ 0.0000000e+00 -5.1315264e-10] [0. 0.] [ 0.0000e+00 -5.1264e-10] [0. 0.] 0.001
[ 0.00000000e+00 -5.13665793e-10] [0. 0.] [0. 0.]
[2.  0.5] [ 0.00000000e+00 -5.13665793e-10] [0. 0.] [0. 0.] [ 0.0000000e+00 -3.2007992e-11] [0. 0.] [ 0.0000000e+00 -3.1976016e-11] 0.002
[0. 0.] [ 0.000e+00 -3.204e-11] [0. 0.]
[2.  0.5] [0. 0.] [ 0.000e+00 -3.204e-11] [ 0.00000000e+00 -4.86211567e+08] [0. 0.] [ 0.00000000e+00 -4.85725841e+08] [0. 0.] 0.003
[ 0.00000000e+00 -4.86697778e+08] [0. 0.] [0. 0.]
[2.  0.5] [ 0.00000000e+00 -4.86697778e+08] [0. 0.] [0. 0.] [ 0.00000000e+00 -1.24968789e+11] [0. 0.] [ 0.00000000e+00 -1.24843945e+11] 0.004


# Two Body, Earth and Sun

Generate iniitial data for two body problem, Sun and Earth, using reduced mass framework.

In [34]:
import math
def InitialData():
    random.seed(a=9001)
    
    #initially use an in plane orbit with random starting locations relative to the x axis
    phi=np.array([0,math.pi])
    orbitangle=np.zeros(2)
    #orbitalradius=np.random.uniform(.1,50,2)
    orbitalradius=np.ones(2)
    #start with circular orbits
    #eccentricity=np.random.uniform(0.,.1,2)
    eccentricity=np.zeros(2)
    #magnitude=np.random.uniform(-20,-30,2) #absolute not apparent maginutde
    #magsun=-26.832
    masssun=1.989*10**30
    lsun=3.828*10**26
    massearth=5.9722*10**24
    #luminosity=lsun*10**(0.4*(magnitude-magsun))
    #masses= invertIMF(luminosity,lsun,masssun) #Initial mass function for Main Sequence
    #masses=np.random.uniform(.7,5.) #replace with IMF
    masses=np.array([masssun,massearth])

    return phi,orbitangle,orbitalradius,eccentricity, masses



In [35]:
import random,numpy as np
initdat=InitialData()

In [36]:
print(initdat)

(array([0.        , 3.14159265]), array([0., 0.]), array([1., 1.]), array([0., 0.]), array([1.9890e+30, 5.9722e+24]))


Transform cylindrical initial data to x,y,z, velocity, acceleration initial data

In [37]:
def getxyuv(initdat):
    phi,orbitangle,orbitalradius,eccentricity, masses=initdat
    #print(orbitalradius, phi, np.cos(phi), np.sin(phi))
    metersperAU=149597870700
    Gconstant=6.408*10**-11
    x0=orbitalradius*np.cos(phi)*metersperAU
    y0=orbitalradius*np.sin(phi)*metersperAU
    z0=np.zeros(2)
    

    
    #start at perihelion for both (eliptical, doesn't generalize to three body)
    #actually start with circular orbit
    ux0=np.zeros(2)*149597870700
    #centrepital force balances gravitational force
    reducedmass=np.zeros(2)
    print(masses)
    for i in np.arange(2):
        j=(i+1)%2 #reverse masses
        reducedmass[i]=masses[i]*masses[j]/np.sum(masses)
    print(reducedmass)
    rphys=np.zeros(len(masses))
    for i in np.arange(2):
        rphys[i]=orbitalradius[i]*metersperAU*reducedmass[i]/masses[i]
    x0=rphys*np.cos(phi)
    y0=rphys*np.sin(phi)
    z0=np.zeros(2)
    print(rphys)
    F=(Gconstant*reducedmass**2/rphys**2)
    print(F)
    #centF=reducedmass*v**2/rphys
    #centF=accel
    v=np.sqrt(Gconstant*reducedmass/rphys)
    print(v)
    ux0=v*np.sin(phi)
    uy0=v*np.cos(phi) #initial data in y only
    #evolve in plane only
    #there is a units problem that needs to be fixed
    #velocity initial conditions are not trivial. 
    uz0=np.zeros(2)
    
    #circular orbit   #a=omega^2 * r #v=omega*r #omega=v/r
    omega=v/rphys
    print(omega)
    omegatrue=np.mean(omega) #shouldaccount for numerical effects
    print(omegatrue)
    a=omegatrue**2*rphys
    print("a",a)
    print(phi)
    ax0=-a*np.cos(phi)
    ay0=-a*np.sin(phi)
    az0=np.zeros(2)
    
    
    return reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0

In [38]:
xyuva=getxyuv(initdat)
print(xyuva)#In SI units
print(xyuva[1][0])
print(xyuva[0][0]/xyuva[0][1])
print(xyuva)

[1.9890e+30 5.9722e+24]
[5.97218207e+24 5.97218207e+24]
[4.49183369e+05 1.49597422e+11]
[1.13276871e+28 1.02126951e+17]
[29188.7795877     50.57847341]
[6.49818796e-02 3.38097227e-10]
0.032490939958166556
a [4.74185445e+02 1.57924190e+08]
[0.         3.14159265]
(array([5.97218207e+24, 5.97218207e+24]), array([ 4.49183369e+05, -1.49597422e+11]), array([0.00000000e+00, 1.83204003e-05]), array([0., 0.]), array([0.00000000e+00, 6.19407656e-15]), array([29188.7795877 ,   -50.57847341]), array([0., 0.]), array([-4.74185445e+02,  1.57924190e+08]), array([-0.00000000e+00, -1.93401354e-08]), array([0., 0.]))
449183.36891987134
1.0
(array([5.97218207e+24, 5.97218207e+24]), array([ 4.49183369e+05, -1.49597422e+11]), array([0.00000000e+00, 1.83204003e-05]), array([0., 0.]), array([0.00000000e+00, 6.19407656e-15]), array([29188.7795877 ,   -50.57847341]), array([0., 0.]), array([-4.74185445e+02,  1.57924190e+08]), array([-0.00000000e+00, -1.93401354e-08]), array([0., 0.]))


Time stepping routine using Euler method and Newtonian differential equation for two body system

In [39]:
def timestep(step,t,dt,reducedmass,xi,yi,zi, vxi, vyi, vzi, axi, ayi, azi):
    xii=np.zeros(np.size(xi))
    vxii=np.zeros(np.size(vx))
    yii=np.zeros(np.size(yi))
    vyii=np.zeros(np.size(vy))
    zii=np.zeros(np.size(vzi))
    vzii=np.zeros(np.size(vzi))
    rii=np.zeros(np.size(xi))
    axii=axi
    ayii=ayi
    azii=azi
    
    for m in np.arange(len(x)):
        #m represents choices of mass
        i=step
        
        xii[m] = xi[m] + dt*vxi[m]
        #print(xii)
        vxii[m] = vxi[m] + dt*axi[m]
        #print(vxii)
        yii[m]= yi[m] + dt*vyi[m]
        vyii[m] = vyi[m] + dt*ayi[m]
        zii[m]= zi[m] + dt*vzi[m]
        vzii[m] = vzi[m] + dt*azi[m]
        rii[m]=np.sqrt(xi[m]**2+yi[m]**2+zi[m]**2)
    
    
    Gconstant=6.408*10**-11
    for k in np.arange(len(rii)):
        for j in np.arange(len(rii)):
            if j!=k:
                rreljk=np.abs((xi[j] - xi[k])**2+(yi[j]-yi[k])**2+(zi[j]-zi[k])**2)**(1./2.)
                axii[j]+=Gconstant*reducedmass[k]*(xi[j]  - xi[k])/rreljk**3
                ayii[j]+=Gconstant*reducedmass[k]*(yi[j]  - yi[k])/rreljk**3
                azii[j]+=Gconstant*reducedmass[k]*(zi[j]  - zi[k])/rreljk**3
    #print(xii)
    return reducedmass, xii,yii,zii,vxii,vyii,vzii,axii,ayii,azii
                    

Run over several time steps and print

In [40]:
dt=0.01*31556926 #seconds per year
numsteps=5
mass,x,y,z,vx,vy,vz,ax,ay,az=xyuva
for i in np.arange(1,numsteps):
    t=0.+i*numsteps*dt
    mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(i,t,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
    print(x,y,vx,vy,ax,ay)
    #print(ay) #forces should be equal and opposite, but in reduced mass framework accelerations are also equal and opposite
    #accelerations should evolve from y to x with time in a sinusoidal manner even in reduced mass framework
    #print(ax)
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(2,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(3,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(4,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 

[ 4.49183369e+05 -1.49597422e+11] [ 9.21108157e+09 -1.59610114e+07] [-1.49638350e+08  4.98360199e+13] [29188.7795877    -50.58457657] [-4.74185445e+02  1.57924190e+08] [-2.09418125e-24 -1.93401354e-08]
[-4.72212629e+13  1.57267158e+19] [ 1.84221631e+10 -3.19239488e+07] [-2.99276700e+08  9.96720398e+13] [29188.7795877    -50.59067972] [-4.74185445e+02  1.57924190e+08] [ 1.04874102e-09 -2.03888765e-08]
[-1.41663790e+14  4.71801476e+19] [ 2.76332447e+10 -4.78888122e+07] [-4.4891505e+08  1.4950806e+14] [29188.77991865   -50.59711382] [-4.74185445e+02  1.57924190e+08] [ 1.04874102e-09 -2.03888765e-08]
[-2.83327580e+14  9.43602954e+19] [ 3.68443264e+10 -6.38557059e+07] [-5.9855340e+08  1.9934408e+14] [29188.7802496    -50.60354792] [-4.74185445e+02  1.57924190e+08] [ 1.04874102e-09 -2.03888765e-08]


Object for storing the orbital differential equation and evolving using implicit RK4 using Newtonian differential equation for two body system

In [76]:
class OrbitDiffEq:
    def __init__(self,reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0):
        self.reducedmass=reducedmass
        self.xi=x0
        self.yi=y0
        self.zi=z0
        self.vxi=ux0
        self.vyi=uy0
        self.vzi=uz0
        self.axi=ax0
        self.ayi=ay0
        self.azi=az0
        #print(self.xi,self.yi,self.vxi,self.vyi,self.axi,self.ayi)
    def dxidt(self,t):
        return self.vxi
    def dyidt(self,t):
        return self.vyi
    def dzidt(self,t):
        return self.vzi
    def dvxidt(self,t):
        #return axi[m]
        axii=self.axi
        rii=np.sqrt(self.xi**2+self.yi**2+self.zi**2)
        Gconstant=6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(len(rii)):
                if j!=k:
                    rreljk=np.abs((self.xi[j] - self.xi[k])**2+(self.yi[j]-self.yi[k])**2+(self.zi[j]-self.zi[k])**2)**(1./2.)
                    axii[j]+=Gconstant*self.reducedmass[k]*(self.xi[j]  - self.xi[k])/rreljk**3
        self.axi=axii
        return axii
    def dvyidt(self,t):
        #return axi[m]
        ayii=self.ayi
        rii=np.sqrt(self.xi**2+self.yi**2+self.zi**2)
        Gconstant=6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(len(rii)):
                if j!=k:
                    rreljk=np.abs((self.xi[j] - self.xi[k])**2+(self.yi[j]-self.yi[k])**2+(self.zi[j]-self.zi[k])**2)**(1./2.)
                    ayii[j]+=Gconstant*self.reducedmass[k]*(self.yi[j]  - self.yi[k])/rreljk**3
        self.ayi=ayii
        return ayii
    def dvzidt(self,t):
        #return axi[m]
        azii=self.azi
        rii=np.sqrt(self.xi**2+self.yi**2+self.zi**2)
        Gconstant=6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(len(rii)):
                if j!=k:
                    rreljk=np.abs((self.xi[j] - self.xi[k])**2+(self.yi[j]-self.yi[k])**2+(self.zi[j]-self.zi[k])**2)**(1./2.)
                    azii[j]+=Gconstant*self.reducedmass[k]*(self.zi[j]  - self.zi[k])/rreljk**3
        self.azi=azii
        return azii
    def updateINTERNAL(self,xii,yii,zii,vxii,vyii,vzii):
        self.xi=xii
        self.yi=yii
        self.zi=zii
        self.vxi=vxii
        self.vyi=vyii
        self.vzi=vzii
        return self
    def update(self,xii,yii,zii,vxii,vyii,vzii,axii,ayii,azii):
        self.xi=xii
        self.yi=yii
        self.zi=zii
        self.vxi=vxii
        self.vyi=vyii
        self.vzi=vzii
        self.axi=axii
        self.ayi=ayii
        self.azi=azii
    def print2D(self):
        print(self.xi,self.yi,self.vxi,self.vyi,self.axi,self.ayi)
        return self
    def list2D(self):
        return self.xi,self.yi,self.vxi,self.vyi,self.axi,self.ayi
    def timestepRK4ODE(self,step,t,dt):

    
        h=dt
        #tnew,ynew, intval=RK4(h,t,y,f)
        #m represents choices of mass
        i=step
        
        tnew,intvalx=RK4implicit(h,t,self.dxidt)
        xii = intvalx
        tnew,intvalvx=RK4implicit(h,t,self.dvxidt)
        vxii=intvalvx
        tnew,intvaly=RK4implicit(h,t,self.dyidt)
        yii = intvaly
        tnew,intvalvy=RK4implicit(h,t,self.dvyidt)
        vyii=intvalvy
        tnew,intvalz=RK4implicit(h,t,self.dzidt)
        zii = intvalz
        tnew,intvalvz=RK4implicit(h,t,self.dvzidt)
        vzii=intvalvz
 
        #print(xii)
        self.updateINTERNAL(xii,yii,zii,vxii,vyii,vzii)
        return reducedmass, xii,yii,zii,vxii,vyii,vzii,self.axi,self.ayi,self.azi

In [77]:
reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0=xyuvaeq
ODEeq= OrbitDiffEq(reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0)
ODEeq.print2D()

[ 0.5 -0.5] [0. 0.] [0. 0.] [ 1. -1.] [-2.  2.] [-0. -0.]


In [72]:
mass,x,y,z,vx,vy,vz,ax,ay,az=xyuva
print(x,y,vx,vy,ax,ay)
print(xyuva)

[ 4.49183369e+05 -1.49597422e+11] [0.00000000e+00 1.83204003e-05] [0.00000000e+00 6.19407656e-15] [29188.7795877    -50.57847341] [-4.74185445e+02  1.57924190e+08] [ 0.00226984 -0.00226986]
(array([5.97218207e+24, 5.97218207e+24]), array([ 4.49183369e+05, -1.49597422e+11]), array([0.00000000e+00, 1.83204003e-05]), array([0., 0.]), array([0.00000000e+00, 6.19407656e-15]), array([29188.7795877 ,   -50.57847341]), array([0., 0.]), array([-4.74185445e+02,  1.57924190e+08]), array([ 0.00226984, -0.00226986]), array([0., 0.]))


Implicit RK4

In [42]:
def RK4implicit(h,t,f): #not a finite difference so no step in y
    k1= h*f(t)
    k2=h*f(t+h/2.)
    k3=h*f(t+h/2.)
    k4=h*f(t+h)
    return t+h, f(t)+1/6.*(k1+2.*k2+2.*k3+k4)




Initialize iniitial conditions 

In [43]:
reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0=xyuva
ODE= OrbitDiffEq(reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0)
ODE.print2D()

[ 4.49183369e+05 -1.49597422e+11] [0.00000000e+00 1.83204003e-05] [0.00000000e+00 6.19407656e-15] [29188.7795877    -50.57847341] [-4.74185445e+02  1.57924190e+08] [ 1.04874102e-09 -2.03888765e-08]


Extract coordinates

In [79]:
mass,x,y,z,vx,vy,vz,ax,ay,az=xyuvaeq
print(x,y,vx,vy,ax,ay)
print(xyuvaeq)

[ 0.5 -0.5] [0. 0.] [0. 0.] [ 1. -1.] [-2.  2.] [-0. -0.]
(array([0.5, 0.5]), array([ 0.5, -0.5]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([ 1., -1.]), array([0., 0.]), array([-2.,  2.]), array([-0., -0.]), array([0., 0.]))


Evolve the differential equation and print

In [45]:
dt=0.001*31556926 #seconds per year
numsteps=5
reducedmass,x,y,z,vx,vy,vz,ax,ay,az=xyuva
ODE.print2D()
for i in np.arange(1,numsteps):
    t=0.+i*numsteps*dt
    reducedmass, x,y,z,vx,vy,vz,ax,ay,az=ODE.timestepRK4ODE(i,t,dt)
    ODE.print2D()
    ODE.update(x,y,z,vx,vy,vz,ax,ay,az)



[ 4.49183369e+05 -1.49597422e+11] [0.00000000e+00 1.83204003e-05] [0.00000000e+00 6.19407656e-15] [29188.7795877    -50.57847341] [-4.74185445e+02  1.57924190e+08] [ 1.04874102e-09 -2.03888765e-08]
[0.0000000e+00 1.9547221e-10] [ 9.21137346e+08 -1.59615172e+06] [-1.49643092e+07  4.98375992e+12] [ 3.30960916e-05 -6.43430654e-04] [-4.74185445e+02  1.57924190e+08] [ 1.04874102e-09 -2.03888765e-08]
[-4.72242562e+11  1.57277127e+17] [  1.04444401 -20.30533698] [-1.49643092e+07  4.98375992e+12] [ 35.46219366 -35.46280399] [-4.74185445e+02  1.57924190e+08] [ 0.00224736 -0.00224738]
[-4.72242562e+11  1.57277127e+17] [ 1119113.28317089 -1119132.54406386] [-1.49643092e+07  4.98375992e+12] [ 70.92210685 -70.92271719] [-4.74185445e+02  1.57924190e+08] [ 0.00224736 -0.00224738]
[-4.72242562e+11  1.57277127e+17] [ 2238154.59982401 -2238173.86071698] [-1.49643092e+07  4.98375992e+12] [ 70.92210685 -70.92271719] [-4.74185445e+02  1.57924190e+08] [ 0.00224736 -0.00224738]


In [46]:
secperyr=31556926

Evolve the differential equation and store in a pandas data frame for later plotting

In [47]:
ODE2= OrbitDiffEq(reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0)



In [48]:
dt=0.01*31556926 #seconds per year
numsteps=5
reducedmass,x,y,z,vx,vy,vz,ax,ay,az=xyuva
datainit=[[0,0.0,x,y,z,vx,vy,vz,ax,ay,az]]
import pandas as pd
columns =['step','time','x' ,'y','z','vx','vy','vz','ax','ay','az']
df = pd.DataFrame(data=datainit, columns=columns)




In [49]:
display(df)

,step,time,x,y,z,vx,vy,vz,ax,ay,az
0,0,0.0,"[449183.36891987134, -149597421516.63107]","[0.0, 1.8320400342103965e-05]","[0.0, 0.0]","[0.0, 6.194076557160162e-15]","[29188.779587697383, -50.57847341349946]","[0.0, 0.0]","[-474.1854448655729, 157924190.42823714]","[0.002247362733938104, -0.0022473820740735355]","[0.0, 0.0]"


In [50]:
for i in np.arange(1,numsteps):
    t=0.+i*numsteps*dt
    reducedmass, x,y,z,vx,vy,vz,ax,ay,az=ODE2.timestepRK4ODE(i,t,dt)
    df.loc[i]=i,t,x,y,z,vx,vy,vz,ax,ay,az
    ODE2.update(x,y,z,vx,vy,vz,ax,ay,az)


In [51]:
display(df)

,step,time,x,y,z,vx,vy,vz,ax,ay,az
0,0,0.0,"[449183.36891987134, -149597421516.63107]","[0.0, 1.8320400342103965e-05]","[0.0, 0.0]","[0.0, 6.194076557160162e-15]","[29188.779587697383, -50.57847341349946]","[0.0, 0.0]","[-474.1854447800713, 157924190.428237]","[0.0022698376326943694, -0.002269856972829801]","[0.0, 0.0]"
1,1,1577846.3,"[0.0, 1.9546663496029367e-09]","[9211110763.572353, -15961062.00550111]","[0.0, 0.0]","[-149638824.11095357, 49836177833728.28]","[709.2008422631583, -709.2069454347248]","[0.0, 0.0]","[-474.1854447800713, 157924190.428237]","[0.0022698376326943694, -0.002269856972829801]","[0.0, 0.0]"
2,2,3155692.6,"[-47221562630787.875, 1.5726815596395868e+19]","[223802694.18520382, -223804620.16464195]","[0.0, 0.0]","[-149638824.0974627, 49836177833728.26]","[712.7470583226017, -712.7531614941684]","[0.0, 0.0]","[-474.1854447800713, 157924190.428237]","[0.0022698376326943694, -0.002269856972829801]","[0.0, 0.0]"
3,3,4733538.9,"[-47221562626530.57, 1.5726815596395866e+19]","[224921774.50909853, -224923700.4885367]","[0.0, 0.0]","[-149638824.0974627, 49836177833728.26]","[716.2932519071464, -716.2993550787132]","[0.0, 0.0]","[-474.1854447800713, 157924190.428237]","[0.0022698376326943694, -0.002269856972829801]","[0.0, 0.0]"
4,4,6311385.2,"[-47221562626530.57, 1.5726815596395866e+19]","[226040847.74058372, -226042773.72002184]","[0.0, 0.0]","[-149638824.0974627, 49836177833728.26]","[716.2932519071464, -716.2993550787132]","[0.0, 0.0]","[-474.1854447800713, 157924190.428237]","[0.0022698376326943694, -0.002269856972829801]","[0.0, 0.0]"


In [52]:
xser=df['x']
xser
xser[1][0]

0.0

Each pandas column has two entries because there are two bodies, the sun and the earth. These need to be split into columns one and two in a new table for each pandas column in the original table. The following routine automates extraction of these entries for each column. 

In [53]:
def mergeStarDataIntoDataFrame(col):
    rawdataser=df[col]
    numstars=len(rawdataser[0])
    columns=np.arange(numstars)
    numentries=len(rawdataser)
    index=np.arange(numentries)
    dfnew=pd.DataFrame(columns=columns,index=index)
    for coln in np.arange(numstars):
        datacollist=[]
        for row in np.arange(numentries):
            datacollist.append(rawdataser[row][coln])
        dfnew[coln]=datacollist
    return dfnew

In [54]:
dfx=mergeStarDataIntoDataFrame('x')

In [55]:
display(dfx)

,0,1
0,4.491834e+05,-1.495974e+11
1,0.000000e+00,1.954666e-09
2,-4.722156e+13,1.572682e+19
3,-4.722156e+13,1.572682e+19
4,-4.722156e+13,1.572682e+19


In [56]:
dfy=mergeStarDataIntoDataFrame('y')

In [57]:
display(dfy)

,0,1
0,0.000000e+00,1.832040e-05
1,9.211111e+09,-1.596106e+07
2,2.238027e+08,-2.238046e+08
3,2.249218e+08,-2.249237e+08
4,2.260408e+08,-2.260428e+08


In [58]:
dfvx=mergeStarDataIntoDataFrame('vx')

In [59]:
display(dfvx)

,0,1
0,0.000000e+00,6.194077e-15
1,-1.496388e+08,4.983618e+13
2,-1.496388e+08,4.983618e+13
3,-1.496388e+08,4.983618e+13
4,-1.496388e+08,4.983618e+13


In [60]:
dfvy=mergeStarDataIntoDataFrame('vy')

In [61]:
display(dfvy)

,0,1
0,29188.779588,-50.578473
1,709.200842,-709.206945
2,712.747058,-712.753161
3,716.293252,-716.299355
4,716.293252,-716.299355


In [62]:
dfax=mergeStarDataIntoDataFrame('ax')

In [63]:
display(dfax)

,0,1
0,-474.185445,1.579242e+08
1,-474.185445,1.579242e+08
2,-474.185445,1.579242e+08
3,-474.185445,1.579242e+08
4,-474.185445,1.579242e+08


In [64]:
dfay=mergeStarDataIntoDataFrame('ay')

In [65]:
display(dfay)

,0,1
0,0.00227,-0.00227
1,0.00227,-0.00227
2,0.00227,-0.00227
3,0.00227,-0.00227
4,0.00227,-0.00227


In [66]:
df['time'].tolist()

[0.0, 1577846.3, 3155692.6, 4733538.9, 6311385.2]

Plots time versus position of Earth, in reduced mass coordinates

In [67]:
from bokeh.plotting import figure, output_notebook, show
p=figure(plot_width=300,plot_height=300,x_axis_label="Time (s)",y_axis_label="y displacement (m), Earth")
show(p)
p.circle(x=np.array(df['time'].tolist()),y=np.array(dfy[1].tolist()))
show(p)
output_notebook()

Loading BokehJS ...